In [ ]:
next(iter(data_iter))


In [1]:
next(iter(data_iter))
import torch
from torch.utils import data
from d2l import torch as d2l

生成数据集

In [3]:
true_w =torch.tensor([2,-3.4])
true_b =4.2
features,labels =d2l.synthetic_data(true_w,true_b,1000)

读取数据集

我们可以调用框架中现有的API来读取数据。我们将features和labels作为API的参数传递，并在实例化数据迭代器对象时指定batch_size。此外，布尔值is_train表示是否希望数据迭代器对象在每个迭代周期内打乱数据。

In [33]:
def load_array(data_arrays, batch_size, is_train=True):
    # 构造一个pytorch数据迭代器
    dataset=data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)


batch_size = 10
data_iter = load_array((features,labels),batch_size)

In [14]:
next(iter(data_iter))

[tensor([[-0.5428,  1.7768],
         [ 1.0280, -0.7536],
         [-1.5071,  1.2666],
         [-0.2598, -1.3152],
         [-1.0523,  0.7510],
         [-0.4316, -0.5905],
         [-0.8895,  0.2526],
         [-0.5312, -0.2714],
         [ 0.9963,  1.5127],
         [ 0.6301, -1.2523]]),
 tensor([[-2.9247],
         [ 8.8279],
         [-3.1251],
         [ 8.1613],
         [-0.4612],
         [ 5.3410],
         [ 1.5765],
         [ 4.0575],
         [ 1.0411],
         [ 9.7228]])]

定义模型

对于标准操作，我们可以使用框架的预定义好的层。这使我们只需关注使用哪些层来构造模型，而不必关注层的实现细节。我们首先定义一个模型变量net，它是一个Sequential类的实例。Sequential类为串联在一起的多个层定义了一个容器。当给定输入数据，Sequential实例将数据传入到第一层，然后将第一层的输出作为第二层的输入，依此类推。在下面的例子中，我们的模型只包含一个层，因此实际上不需要Sequential。但是由于以后几乎所有的模型都是多层的，在这里使用Sequential会让你熟悉标准的流水线。

在PyTorch中，全连接层在Linear类中定义。值得注意的是，我们将两个参数传递到nn.Linear中。第一个指定输入特征形状，即2，第二个指定输出特征形状，输出特征形状为单个标量，因此为1。

In [16]:
from torch import nn

net = nn.Sequential(nn.Linear(2,1))

初始化模型参数

在使用net之前，我们需要初始化模型参数。如在线性回归模型中的权重和偏置。 深度学习框架通常有预定义的方法来初始化参数。 在这里，我们指定每个权重参数应该从均值为0、标准差为0.01的正态分布中随机采样，偏置参数将初始化为零。

正如我们在构造nn.Linear时指定输入和输出尺寸一样。现在我们直接访问参数以设定初始值。我们通过net[0]选择网络中的第一个图层，然后使用weight.data和bias.data方法访问参数。然后使用替换方法normal_和fill_来重写参数值

In [19]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)
net[0].bias.data.fill_(0)

tensor([0.])

计算均方误差使用的是MSELoss类，也称为平方\(L_2\)范数。默认情况下，它返回所有样本损失的平均值。

In [20]:
loss = nn.MSELoss()

定义优化算法

小批量随机梯度下降算法是一种优化神经网络的标准工具，PyTorch在optim模块中实现了该算法的许多变种。当我们实例化SGD实例时，我们要指定优化的参数（可通过net.parameters()从我们的模型中获得）以及优化算法所需的超参数字典。小批量随机梯度下降只需要设置lr值，这里设置为0.03。

In [21]:
trainer =torch.optim.SGD(net.parameters(),lr=0.03)

训练

通过深度学习框架的高级API来实现我们的模型只需要相对较少的代码。 我们不必单独分配参数、不必定义我们的损失函数，也不必手动实现小批量随机梯度下降。 当我们需要更复杂的模型时，高级API的优势将大大增加。 当我们有了所有的基本组件，训练过程代码与我们从零开始实现时所做的非常相似。

回顾一下：在每个迭代周期里，我们将完整遍历一次数据集（train_data），不停地从中获取一个小批量的输入和相应的标签。对于每一个小批量，我们会进行以下步骤:

通过调用net(X)生成预测并计算损失l（正向传播）。

通过进行反向传播来计算梯度。

通过调用优化器来更新模型参数。

为了更好的衡量训练效果，我们计算每个迭代周期后的损失，并打印它来监控训练过程。

In [24]:
num_epochs=5
for epoch in range(num_epochs):
    for X,y in data_iter:
        l =loss(net(X),y)
        #这一步是清除上一轮的trainer的梯度 不让他累加到这一批数据上
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l =loss(net(features),labels)
    print(f'epoch:{epoch+1},loss:{l:f}')

epoch:1,loss:0.000102
epoch:2,loss:0.000102
epoch:3,loss:0.000102
epoch:4,loss:0.000102
epoch:5,loss:0.000103


下面我们比较生成数据集的真实参数和通过有限数据训练获得的模型参数。 要访问参数，我们首先从net访问所需的层，然后读取该层的权重和偏置。 正如在从零开始实现中一样，我们估计得到的参数与生成数据的真实参数非常接近。

In [26]:
w = net[0].weight.data
print('w的估计误差：', true_w-w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b-b)


w的估计误差： tensor([-0.0003,  0.0003])
b的估计误差： tensor([-0.0007])


小结
我们可以使用PyTorch的高级API更简洁地实现模型。

在PyTorch中，data模块提供了数据处理工具，nn模块定义了大量的神经网络层和常见损失函数。

我们可以通过_结尾的方法将参数替换，从而初始化参数。